In [2]:
import glob
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# 폴더 경로 지정
folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017'

In [3]:
# 랜섬웨어 클래스별 파일에 있는 샘플 개수를 확인하기 위한 딕셔너리
samples_per_class = {}

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

# 모든 파일 로드 및 샘플 개수 확인
for folder in ransomware_folder_list:
    ransomware_class = folder.split('\\')[-1]  # 랜섬웨어 클래스명 추출

    file_paths = glob.glob(folder + '\\*.csv')
    total_samples = 0  # 클래스별 전체 샘플 개수 초기화
    for file_path in file_paths:
        data = pd.read_csv(file_path)
        total_samples += data.shape[0]  # 데이터프레임의 행 수 / 샘플 개수 더하기

    samples_per_class[ransomware_class] = total_samples

# 클래스별 샘플 개수 출력
for ransomware_class, num_samples in samples_per_class.items():
    print(f"{ransomware_class} 클래스의 샘플 개수: {num_samples}")

Charger 클래스의 샘플 개수: 39551
Jisut 클래스의 샘플 개수: 25672
Koler 클래스의 샘플 개수: 44555
Lockerpin 클래스의 샘플 개수: 25307
Pletor 클래스의 샘플 개수: 4715
PornDroid 클래스의 샘플 개수: 46082
RansomBO 클래스의 샘플 개수: 39859
Simplocker 클래스의 샘플 개수: 36340
SVpeng 클래스의 샘플 개수: 54161
WannaLocker 클래스의 샘플 개수: 32701


In [4]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 200,000개 무작위 샘플 추출
num_samples_benign = 200000
labels = []

if len(benign_data) > num_samples_benign:
    feature_benign_subset = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    feature_benign_subset = benign_data.copy()

# Benign 데이터의 라벨을 labels 리스트에 추가
labels.extend(feature_benign_subset[' Label'].tolist())

print("특성추출 benign_subset의 샘플 개수:", len(feature_benign_subset))

특성추출 benign_subset의 샘플 개수: 200000


In [5]:
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')
all_ransomware_data = pd.DataFrame()

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    
    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        all_ransomware_data = pd.concat([all_ransomware_data, data], ignore_index=True)

# 전체 랜섬웨어 데이터에서 200,000개 샘플 무작위로 추출
if len(all_ransomware_data) > 200000:
    feature_ransomware_subset = all_ransomware_data.sample(n=200000, random_state=42, replace=False)
else:
    feature_ransomware_subset = all_ransomware_data.copy()

# 랜섬웨어 데이터의 라벨을 labels 리스트에 추가
labels.extend(feature_ransomware_subset[' Label'].tolist())

print("특성추출 ransomware_subset의 샘플 개수:", len(feature_ransomware_subset))

특성추출 ransomware_subset의 샘플 개수: 200000


In [6]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
feature_dataset = pd.concat([feature_benign_subset, feature_ransomware_subset], ignore_index=True)

print("최종 특성추출 데이터세트의 샘플 개수:", len(feature_dataset))

최종 특성추출 데이터세트의 샘플 개수: 400000


In [ ]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = feature_dataset[target_variable]  # 타깃 변수
X = feature_dataset.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거 (예시: 'Flow ID', ' Timestamp', ' Source IP', ' Destination IP' 특성 제거)
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# 각 열(feature)에 Min-Max 스케일링 적용
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# chi2를 사용하여 특성 선택 (하위 36개의 특성 선택)
num_features_to_select = 36
selector = SelectKBest(score_func=chi2, k=num_features_to_select)
X_selected = selector.fit_transform(X_scaled, y)

# 선택된 특성들의 인덱스 추출
selected_feature_indices = selector.get_support(indices=True)

# 선택된 특성들의 컬럼명 추출
selected_feature_names = X.columns[selected_feature_indices]

# 최종 선택된 특성들의 데이터프레임 생성
X_final = X[selected_feature_names]

# 결과 출력
print("선택된 특성들의 컬럼명:")
print(selected_feature_names)
print("\n최종 선택된 특성들의 데이터프레임:")
print(X_final.head())

In [8]:
# 특성들의 가중치(Chi-square 통계량) 확인
chi2_scores = selector.scores_[selected_feature_indices]

# 특성들의 가중치를 기준으로 오름차순 정렬
sorted_indices = chi2_scores.argsort()
sorted_features = selected_feature_names[sorted_indices]
sorted_chi2_scores = chi2_scores[sorted_indices]

# 특성들의 가중치와 순위 출력
print("특성들의 순위와 가중치:")
for i, (feature, score) in enumerate(zip(sorted_features, sorted_chi2_scores), 1):
    print(f"{i}. {feature}: {score}")

특성들의 순위와 가중치:
1.  Fwd Packet Length Max: 68.51133868993816
2.  Idle Std: 71.12354356709263
3. Total Length of Fwd Packets: 73.42559166162557
4.  Subflow Fwd Bytes: 73.42559166162557
5.  Active Min: 83.28418721520666
6.  Bwd IAT Max: 88.8963958229467
7.  Bwd Packet Length Std: 114.4627489105487
8. Bwd IAT Total: 127.52163666858264
9. Active Mean: 177.51414388595342
10.  Flow IAT Std: 185.62858152753807
11.  Init_Win_bytes_backward: 187.01606362293234
12. Fwd Packets/s: 223.70671692479854
13.  Fwd IAT Min: 224.95806560317286
14.  Flow Packets/s: 231.97393796382588
15.  Fwd IAT Mean: 238.4838937395082
16.  Fwd IAT Std: 251.5386000987008
17.  Flow IAT Min: 262.0408382976124
18.  Flow IAT Mean: 283.16758689898757
19.  Source Port: 298.84531536111183
20.  Idle Max: 303.9776781317766
21. Idle Mean: 304.2946892955119
22.  Idle Min: 312.0969996165415
23.  Active Max: 359.15956346435263
24.  Flow IAT Max: 379.90025003727374
25. Fwd PSH Flags: 393.5098629097659
26.  SYN Flag Count: 393.5098629097

In [9]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 160,000개 무작위 샘플 추출
num_samples_benign = 160000
labels = []

if len(benign_data) > num_samples_benign:
    train_benign_subset = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    train_benign_subset = benign_data.copy()

# Benign 데이터의 라벨을 labels 리스트에 추가
labels.extend(train_benign_subset[' Label'].tolist())

print("train_benign_subset의 샘플 개수:", len(train_benign_subset))

train_benign_subset의 샘플 개수: 160000


In [10]:
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')
all_ransomware_data = pd.DataFrame()

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    
    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        all_ransomware_data = pd.concat([all_ransomware_data, data], ignore_index=True)

# 전체 랜섬웨어 데이터에서 160,000개 샘플 무작위로 추출
if len(all_ransomware_data) > 160000:
    train_ransomware_subset = all_ransomware_data.sample(n=160000, random_state=42, replace=False)
else:
    train_ransomware_subset = all_ransomware_data.copy()

# 랜섬웨어 데이터의 라벨을 labels 리스트에 추가
labels.extend(train_ransomware_subset[' Label'].tolist())

print("train_ransomware_subset의 샘플 개수:", len(train_ransomware_subset))

train_ransomware_subset의 샘플 개수: 160000


In [12]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
train_dataset = pd.concat([train_benign_subset, train_ransomware_subset], ignore_index=True)

print("최종 train 데이터세트의 샘플 개수:", len(train_dataset))

최종 train 데이터세트의 샘플 개수: 320000


In [13]:
from sklearn.preprocessing import MinMaxScaler

# 선택된 36가지 특성에 해당하는 열만 추출
selected_feature_columns = X_final.columns
selected_feature_values = train_dataset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화
scaler = MinMaxScaler()
selected_feature_values_normalized = scaler.fit_transform(selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환
X_normalized = pd.DataFrame(selected_feature_values_normalized, columns=selected_feature_columns)

# 결과 출력
print("정규화된 특성들의 데이터프레임:")
print(X_normalized.head())

정규화된 특성들의 데이터프레임:
    Source Port   Destination Port   Flow Duration  \
0      0.592166           0.000811        0.000794   
1      0.758065           0.001224        0.002154   
2      0.564211           0.006780        0.085388   
3      0.739433           0.001224        0.004174   
4      0.720954           0.006780        0.024747   

   Total Length of Fwd Packets   Fwd Packet Length Max  \
0                     0.000004                0.001924   
1                     0.000000                0.000000   
2                     0.000058                0.025503   
3                     0.000101                0.048244   
4                     0.000005                0.001135   

    Bwd Packet Length Std   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
0                0.000000     1.048297e-05        0.000795       0.000000   
1                0.000000     3.861314e-06        0.002155       0.000000   
2                0.059695     4.796322e-07        0.009495       0.039757   


In [14]:
# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환
num_rows = 6
num_columns = 6
X_final_reshaped = X_normalized.values.reshape(-1, num_rows, num_columns)

# 결과 출력
print("6x6 크기의 2차원 행렬:")
print(X_final_reshaped[0])  # 첫 번째 샘플에 해당하는 6x6 행렬 출력

6x6 크기의 2차원 행렬:
[[5.92165899e-01 8.11178964e-04 7.94301774e-04 4.00783933e-06
  1.92383583e-03 0.00000000e+00]
 [1.04829678e-05 7.94961759e-04 0.00000000e+00 7.94961759e-04
  7.95120099e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.24146129e-06 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.00783933e-06 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [15]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\36_feature_binary_CNN\Train_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# 이미지 확대 비율 설정
resize_factor = 10  # 10배 확대

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_final_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_final_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대
    
    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 160000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [16]:
import os
import cv2
import numpy as np

# 라벨 목록 및 해당 라벨에 대한 정수 값 지정
label_mapping = {
    "Benign": 0,
    "Ransomware": 1
}

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\36_feature_binary_CNN\Train_Image\\'

X_data = []
y_labels = []

# 모든 라벨마다 폴더를 순회합니다.
for label_str, label_int in label_mapping.items():
    folder_path = os.path.join(image_folder, label_str)  # 해당 라벨의 폴더 경로
    
    # 해당 폴더 내의 모든 이미지 파일을 순차적으로 처리
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # 16-bit 이미지 로딩
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:  # 이미지 로딩 실패
            print(f"Failed to load image at path: {image_path}")
            continue
        
        X_data.append(image)
        y_labels.append(label_int)

X_data = np.array(X_data, dtype=np.uint16)  # uint16 형태로 이미지 데이터 저장
y_labels = np.array(y_labels)

In [17]:
# train_benign_subset에 포함되지 않은 데이터 선택
validation_data_candidate = benign_data.drop(train_benign_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_validation = 20000
if len(validation_data_candidate) > num_samples_validation:
    validation_benign_subset = validation_data_candidate.sample(n=num_samples_validation, random_state=42)
else:
    validation_benign_subset = validation_data_candidate.copy()

print("validation_benign_subset의 샘플 개수:", len(validation_benign_subset))

validation_benign_subset의 샘플 개수: 20000


In [18]:
# train_ransomware_subset에 포함되지 않은 랜섬웨어 데이터 선택
validation_ransomware_candidate = all_ransomware_data.drop(train_ransomware_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_validation_ransomware = 20000
if len(validation_ransomware_candidate) > num_samples_validation_ransomware:
    validation_ransomware_subset = validation_ransomware_candidate.sample(n=num_samples_validation_ransomware, random_state=42)
else:
    validation_ransomware_subset = validation_ransomware_candidate.copy()

print("validation_ransomware_subset의 샘플 개수:", len(validation_ransomware_subset))

validation_ransomware_subset의 샘플 개수: 20000


In [20]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
validation_dataset = pd.concat([validation_benign_subset, validation_ransomware_subset], ignore_index=True)

print("최종 validation 데이터세트의 샘플 개수:", len(validation_dataset))

최종 validation 데이터세트의 샘플 개수: 40000


In [21]:
# 선택된 36가지 특성에 해당하는 열만 추출 (Validation 데이터셋)
val_selected_feature_values = validation_dataset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 (Validation 데이터셋)
val_selected_feature_values_normalized = scaler.transform(val_selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환 (Validation 데이터셋)
X_val_normalized = pd.DataFrame(val_selected_feature_values_normalized, columns=selected_feature_columns)

# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환 (Validation 데이터셋)
X_val_reshaped = X_val_normalized.values.reshape(-1, num_rows, num_columns)

In [22]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\36_feature_binary_CNN\Validation_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# 이미지 확대 비율 설정
resize_factor = 10  # 10배 확대

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_val_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_val_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대
    
    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 20000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [23]:
import os
import cv2
import numpy as np

# 라벨 목록 및 해당 라벨에 대한 정수 값 지정
label_mapping = {
    "Benign": 0,
    "Ransomware": 1
}

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\36_feature_binary_CNN\Validation_Image\\'  # Modified path to "Validation_Image"

X_val_data = []  # Changed variable name to X_val_data for clarity
y_val_labels = []  # Changed variable name to y_val_labels for clarity

# 모든 라벨마다 폴더를 순회합니다.
for label_str, label_int in label_mapping.items():
    folder_path = os.path.join(image_folder, label_str)  # 해당 라벨의 폴더 경로
    
    # 해당 폴더 내의 모든 이미지 파일을 순차적으로 처리
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # 16-bit 이미지 로딩
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:  # 이미지 로딩 실패
            print(f"Failed to load image at path: {image_path}")
            continue
        
        X_val_data.append(image)  # Changed variable name
        y_val_labels.append(label_int)  # Changed variable name

X_val_data = np.array(X_val_data, dtype=np.uint16)  # uint16 형태로 이미지 데이터 저장, changed variable name
y_val_labels = np.array(y_val_labels)  # Changed variable name

In [24]:
# train_benign_subset 및 validation_benign_subset에 포함되지 않은 데이터 선택
test_data_candidate = benign_data.drop(train_benign_subset.index).drop(validation_benign_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_test = 20000
if len(test_data_candidate) > num_samples_test:
    test_benign_subset = test_data_candidate.sample(n=num_samples_test, random_state=42)
else:
    test_benign_subset = test_data_candidate.copy()

print("test_benign_subset의 샘플 개수:", len(test_benign_subset))

test_benign_subset의 샘플 개수: 20000


In [25]:
# validation_ransomware_subset에 포함되지 않은 랜섬웨어 데이터 선택
test_ransomware_candidate = validation_ransomware_candidate.drop(validation_ransomware_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_test_ransomware = 20000
if len(test_ransomware_candidate) > num_samples_test_ransomware:
    test_ransomware_subset = test_ransomware_candidate.sample(n=num_samples_test_ransomware, random_state=42)
else:
    test_ransomware_subset = test_ransomware_candidate.copy()

print("test_ransomware_subset의 샘플 개수:", len(test_ransomware_subset))

test_ransomware_subset의 샘플 개수: 20000


In [27]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
test_dataset = pd.concat([test_benign_subset, test_ransomware_subset], ignore_index=True)

print("최종 test 데이터세트의 샘플 개수:", len(test_dataset))

최종 test 데이터세트의 샘플 개수: 40000


In [28]:
# 선택된 36가지 특성에 해당하는 열만 추출 (Test 데이터셋)
test_selected_feature_values = test_dataset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 (Test 데이터셋)
test_selected_feature_values_normalized = scaler.transform(test_selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환 (Test 데이터셋)
X_test_normalized = pd.DataFrame(test_selected_feature_values_normalized, columns=selected_feature_columns)

# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환 (Test 데이터셋)
X_test_reshaped = X_test_normalized.values.reshape(-1, num_rows, num_columns)

In [29]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\36_feature_binary_CNN\Test_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# 이미지 확대 비율 설정
resize_factor = 10  # 10배 확대

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_test_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_test_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대
    
    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 20000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [30]:
import os
import cv2
import numpy as np

# 라벨 목록 및 해당 라벨에 대한 정수 값 지정
label_mapping = {
    "Benign": 0,
    "Ransomware": 1
}

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\36_feature_binary_CNN\Test_Image\\'  # Modified path to "Test_Image"

X_test_data = []  # Changed variable name to X_test_data for clarity
y_test_labels = []  # Changed variable name to y_test_labels for clarity

# 모든 라벨마다 폴더를 순회합니다.
for label_str, label_int in label_mapping.items():
    folder_path = os.path.join(image_folder, label_str)  # 해당 라벨의 폴더 경로
    
    # 해당 폴더 내의 모든 이미지 파일을 순차적으로 처리
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # 16-bit 이미지 로딩
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:  # 이미지 로딩 실패
            print(f"Failed to load image at path: {image_path}")
            continue
        
        X_test_data.append(image)  # Changed variable name
        y_test_labels.append(label_int)  # Changed variable name

X_test_data = np.array(X_test_data, dtype=np.uint16)  # uint16 형태로 이미지 데이터 저장, changed variable name
y_test_labels = np.array(y_test_labels)  # Changed variable name